In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## STEP 1 : Data Collection

In [2]:
filepath_House_Price = r"S3_Part2_FE_VIF.csv"

In [3]:
houses = pd.read_csv(filepath_House_Price , header = 0 ,index_col= 0)

In [4]:
houses.head()

,price,crime_rate,resid_area,air_qual,room_num,age,teachers,poor_prop,airport,n_hos_beds,n_hot_rooms,waterbody,rainfall,dist
0,24.0,-5.064036,32.31,0.538,6.575,65.2,24.7,1.605430,YES,5.480,11.1920,River,23,4.0875
1,21.6,-3.600502,37.07,0.469,6.421,78.9,22.2,2.212660,NO,7.332,12.1728,Lake,42,4.9675
2,34.7,-3.601235,37.07,0.469,7.185,61.1,22.2,1.393766,NO,7.394,101.1200,None,38,4.9675
3,33.4,-3.430523,32.18,0.458,6.998,45.8,21.3,1.078410,YES,9.268,11.2672,Lake,45,6.0650
4,36.2,-2.672924,32.18,0.458,7.147,54.2,21.3,1.673351,NO,8.824,11.2896,Lake,55,6.0625


In [5]:
houses.shape

(506, 14)

## STEP 2 : Package importing 

In [6]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

# from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder

from sklearn.pipeline import Pipeline                
from sklearn.compose import ColumnTransformer

from sklearn.model_selection import train_test_split

## STEP 3 : Pipeline

In [7]:
df = houses.copy()

In [8]:
def num_cat_col_list(DF) : 
    
    from pandas.api.types import is_string_dtype, is_numeric_dtype
   
    num_list = []
    cat_list = []

    for column in DF:

        if is_numeric_dtype(DF[column]):
            num_list.append(column)

        elif is_string_dtype(DF[column]):
            cat_list.append(column)

    return (num_list , cat_list)

In [9]:
num_list , cat_list = num_cat_col_list(DF = df)

print("Numerical column : ",num_list)
print("Categorical column : ",cat_list)

Numerical column :  ['price', 'crime_rate', 'resid_area', 'air_qual', 'room_num', 'age', 'teachers', 'poor_prop', 'n_hos_beds', 'n_hot_rooms', 'rainfall', 'dist']
Categorical column :  ['airport', 'waterbody']


### Making changes to num_feat and cat_feat list

In [10]:
# num_list

num_feat = num_list.copy()
num_feat.remove('price')

cat_feat = cat_list.copy()

In [11]:
def pipeline_trans(DF,num_feat ,cat_feat):
    
    # Separating Independent and dependent variables
    X  = DF.drop(labels = ['price' ], axis = 1)
    y  = DF['price']
    
    # Train-Test Split
    X_train,X_test,y_train,y_test = train_test_split(X,y , random_state= 100 ,test_size = 0.2 )
    
    # Pipeline
    ohe = OneHotEncoder()
    
    num_pipeline = Pipeline([('Imputer' , SimpleImputer(strategy='median')) ,
                             ("scaling", StandardScaler())
                            ])
    
    cat_pipeline = Pipeline([('Nominal Encoding' ,ohe)])
    
    pipeline = ColumnTransformer([("Num pipeline" , num_pipeline , num_feat),
                                ("Nominal pipeline" , cat_pipeline , cat_feat)
                                ])
    
    # Fit and Transformation
    X_train_tr = pipeline.fit_transform(X_train)
    X_test_tr =  pipeline.transform(X_test)
    
    # Output Columns after tranformation
    out_cols = []

    i= 0  # counter

    for col in pipeline.get_feature_names_out():
        out_cols.append(col.split("__")[-1])
        #out_cols[i] = out_cols[i].split("_")[-1]

        i += 1
    # print(out_cols)
    
    # Conversion to train and test Dataframes
    X_train_tr_df = pd.DataFrame(X_train_tr, columns= out_cols)
    X_test_tr_df = pd.DataFrame(X_test_tr , columns = out_cols)
    
    y_train.reset_index(drop = True , inplace = True)
    y_test.reset_index(drop = True , inplace = True)
    
    df_train_trans = pd.concat([X_train_tr_df,y_train] ,axis = 1 )
    df_test_trans = pd.concat([X_test_tr_df,y_test] ,axis = 1 )
    
    return df_train_trans,df_test_trans

In [12]:
df_train_tr ,df_test_tr = pipeline_trans(DF = df , num_feat = num_feat , cat_feat = cat_feat)

In [13]:
df_train_tr.head(3)

,crime_rate,resid_area,air_qual,room_num,age,teachers,poor_prop,n_hos_beds,n_hot_rooms,rainfall,dist,airport_NO,airport_YES,waterbody_Lake,waterbody_Lake and River,waterbody_None,waterbody_River,price
0,1.750941,1.035280,1.019549,-0.074765,1.128863,-0.813975,1.194251,-0.019931,-0.356733,0.562485,-1.174428,1.0,0.0,0.0,1.0,0.0,0.0,10.2
1,0.283472,-0.161802,-0.077387,-0.218185,-0.552953,0.039554,-0.958566,0.209591,0.350918,0.562485,-0.563210,0.0,1.0,0.0,0.0,0.0,1.0,22.1
2,0.489127,1.251338,0.449488,0.947639,1.036220,1.794031,-1.399167,0.198726,0.031860,1.457145,-0.934830,0.0,1.0,0.0,0.0,1.0,0.0,41.3


In [14]:
df_test_tr.head(3)

,crime_rate,resid_area,air_qual,room_num,age,teachers,poor_prop,n_hos_beds,n_hot_rooms,rainfall,dist,airport_NO,airport_YES,waterbody_Lake,waterbody_Lake and River,waterbody_None,waterbody_River,price
0,-1.148833,-1.385161,-1.286608,1.417661,-1.069612,2.789815,-0.789224,1.533755,0.195398,0.318486,1.721522,1.0,0.0,1.0,0.0,0.0,0.0,34.6
1,0.009692,-0.701948,-0.422879,0.392417,-1.671787,0.513737,-1.731375,-0.409711,0.363912,0.643817,-0.203814,0.0,1.0,0.0,0.0,1.0,0.0,31.5
2,-1.062466,0.134549,0.173094,-0.221025,0.298645,-1.193321,-0.262952,-0.897276,0.003244,-1.552167,-0.734351,0.0,1.0,0.0,1.0,0.0,0.0,20.6


In [15]:
D_train = df_train_tr.copy()
D_test = df_test_tr.copy()

## STEP 5 : Export the DF as a .csv file.

In [16]:
D_train.to_csv("S3_Part3_pipeline_pipeline_Train_DS.csv")

In [17]:
D_test.to_csv("S3_Part3_pipeline_pipeline_Test_DS.csv")